# Working with text corpora

Your text data usually comes in the form of (long) plain text strings that are stored in one or several files on disk. We can load and transform this data into a [`Corpus`](api.rst#TODO) object so that we can perform all kinds of operations that are implemented as *corpus functions* in tmtoolkit. The `Corpus` class itself resembles a [Python dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) with some additional functionality.

Let's import the `Corpus` class first:

In [1]:
from tmtoolkit.corpus import Corpus

## Loading text data

Several methods are implemented to load text data from different sources:

- load built-in datasets
- load plain text files (".txt files")
- load folder(s) with plain text files
- load a tabular (i.e. CSV or Excel) file containing document IDs and texts
- load a ZIP file containing plain text or tabular files

We can create a `Corpus` object directly by immediately loading a dataset using one of the `Corpus.from_...` methods. This is what we've done when we used `corp = Corpus.from_builtin_corpus('en-News100')` in the [previous chapter](getting_started.ipynb).

Let's load a folder with example documents. Make sure that the path is relative to the current working directory. The data for these examples can be downloaded from [GitHub](https://github.com/WZBSocialScienceCenter/tmtoolkit/tree/master/doc/source/data). 


<div class="alert alert-info">

### Note: Rich text documents

If you want to work with "rich text documents", i.e. formatted, non-plain text sources such as PDFs, Word documents, HTML files, etc. you must convert them to one of the supported formats first. For example you can use the [pdftotext](https://www.mankier.com/1/pdftotext) command from the Linux package `poppler-utils` to convert from PDF to plain text files or [pandoc](https://pandoc.org/) to convert from Word or HTML to plain text.

</div>

In [2]:
corp = Corpus.from_folder('data/corpus_example', language='en')
corp

<Corpus [3 documents  / language "en"]>

Again, we can have a look which document labels were created and print one sample document:

In [3]:
corp.doc_labels

['sample1', 'sample2', 'sample3']

The [`corpus_summary`](api.rst#TODO) and [`print_summary`](api.rst#TODO) functions are very helpful to get a first overview of a loaded corpus:

In [4]:
from tmtoolkit.corpus import print_summary

print_summary(corp)

Corpus with 3 documents in English
> sample1 (25 tokens): This is the first example file . ☺ We showcase NER...
> sample2 (20 tokens): Here comes the second example .    This one contai...
> sample3 (36 tokens): And here we go with the third and final example fi...
total number of tokens: 81 / vocabulary size: 53


<div class="alert alert-info">

### Side note: Corpus functions

The [`corpus_summary`](api.rst#TODO) and [`print_summary`](api.rst#TODO) functions are examples of *corpus functions*. All corpus functions accept a `Corpus` object as first argument and operate on it. A corpus function may retrieve information from a corpus and/or modify it. Most functions in the `tmtoolkit.corpus` module are corpus functions.

<div/>

Another option is to create a `Corpus` object and adding further documents using the `corpus_add_...` functions. Here we create an empty `Corpus` and then add documents via [`corpus_add_files`](api.rst#TODO) which is another example of a corpus function (one that modifies a `Corpus` object). It takes a `Corpus` object and one or more paths to raw text files.

In [5]:
corp = Corpus(language='en')
print_summary(corp)

Corpus with 0 document in English
total number of tokens: 0 / vocabulary size: 0


In [6]:
from tmtoolkit.corpus import corpus_add_files

corpus_add_files(corp, 'data/corpus_example/sample1.txt')
print_summary(corp)

Corpus with 1 document in English
> data_corpus_example-sample1 (25 tokens): This is the first example file . ☺ We showcase NER...
total number of tokens: 25 / vocabulary size: 24


Note that this time the document label is different. Its prefixed by a normalized version of the path to the document. We can alter the `doc_label_fmt` argument of [Corpus.add_files()](api.rst#tmtoolkit.corpus.Corpus.add_files) in order to control how document labels are generated. But at first, let's remove the previously loaded document from the corpus. Since a `Corpus` instance behaves like a Python `dict`, we can use `del`:

In [7]:
del corp['data_corpus_example-sample1']
print_summary(corp)

Corpus with 0 document in English
total number of tokens: 0 / vocabulary size: 0


Now we use a modified `doc_label_fmt` paramater value to generate document labels only from the file name and not from the full path to the document. We also load three files now:

In [8]:
corpus_add_files(corp, ['data/corpus_example/sample1.txt',
                        'data/corpus_example/sample2.txt',
                        'data/corpus_example/sample3.txt'],
                 doc_label_fmt='{basename}')
print_summary(corp)


Corpus with 3 documents in English
> sample1 (25 tokens): This is the first example file . ☺ We showcase NER...
> sample2 (20 tokens): Here comes the second example .    This one contai...
> sample3 (36 tokens): And here we go with the third and final example fi...
total number of tokens: 81 / vocabulary size: 53


As noted in the beginning, there are more `corpus_add_...` and `Corpus.from_...` functions/methods to load text data from different sources. See the [corpus module API](api.rst#TODO) for details.

<div class="alert alert-info">

### Note

Please be aware of the difference of the `corpus_add_...` and `Corpus.from_...` functions/methods: The former *modifies* a given `Corpus` object, whereas the latter *creates* a new `Corpus` object.

</div>

## Configuring the NLP pipeline, parallel processing and more via Corpus parameters

When initializing a `Corpus`, you can pass several arguments. You *must* at least provide either the `language`, `language_model` or `spacy_instance` argument. The simplest is to just pass the `language` as two-letter ISO 639-1 language code. A respective SpaCy language model will then be automatically selected depending on that language code and the NLP pipeline features that you require (more on that next). Alternatively, you set the SpaCy language model to be loaded via `language_model`. Finally, if you already loaded a [SpaCy pipeline instance](https://spacy.io/api/language) yourself you can also pass it via the `spacy_instance` parameter.

You can specify the features, i.e. the components, of the NLP pipeline using the `load_features` parameter. This only applies if you don't provide your own pipeline instance via `spacy_instance`. It determines the pipeline's capabilities, i.e. if POS-tags, named entities, sentences, etc. are recognized. By default, tmtoolkit will load and enable all components but the NER component. The more components are enabled, the slower the text processing works and the more memory it uses. So it makes sense to only enable pipelines that you actually use. See the [SpaCy documentation](https://spacy.io/models#design) for which pipelines are implemented (note that this also depends on the language and language model).

Let's create a corpus with only a minimal pipeline. If we set `load_features` to an empty sequence, only a basic tokenizing pipeline is run.

In [9]:
corp_minimal = Corpus.from_folder('data/corpus_example', language='en', load_features=[])

We can compare the token table output of the minimal pipeline with the default pipeline used in the previously loaded `corp` object:

In [10]:
from tmtoolkit.corpus import tokens_table

tokens_table(corp_minimal)

,doc,position,token,is_punct,is_stop,like_num
0,sample1,0,This,False,True,False
1,sample1,1,is,False,True,False
2,sample1,2,the,False,True,False
3,sample1,3,first,False,True,True
4,sample1,4,example,False,False,False
...,...,...,...,...,...,...
76,sample3,31,third,False,True,True
77,sample3,32,and,False,True,False
78,sample3,33,final,False,False,False
79,sample3,34,paragraph,False,False,False


In [11]:
tokens_table(corp)

,doc,position,token,is_punct,is_stop,lemma,like_num,pos,tag
0,sample1,0,This,False,True,this,False,PRON,DT
1,sample1,1,is,False,True,be,False,AUX,VBZ
2,sample1,2,the,False,True,the,False,DET,DT
3,sample1,3,first,False,True,first,True,ADJ,JJ
4,sample1,4,example,False,False,example,False,NOUN,NN
...,...,...,...,...,...,...,...,...,...
76,sample3,31,third,False,True,third,True,ADJ,JJ
77,sample3,32,and,False,True,and,False,CCONJ,CC
78,sample3,33,final,False,False,final,False,ADJ,JJ
79,sample3,34,paragraph,False,False,paragraph,False,NOUN,NN


As we can see, the minimal pipeline produces less information: The lemmata and POS tags are missing from the table for the corpus using the minimal pipeline. Also, if we'd wanted sentences, we'd get an error since sentences recognition is not enabled in the minimal pipeline:

```python
tokens_table(corp_minimal, sentences=True)
# results in ValueError: sentence numbers requested, but sentence borders not set; Corpus documents probably not parsed with sentence recognition
```

You can also use the default pipeline as basis and add new components via the `add_features` parameter. Here, we add the NER component for finding out named entities like persons. The table then shows a new column `ent_type` with `"PERSON"` entries next to names:

In [12]:
corp_ner = Corpus.from_folder('data/corpus_example', language='en', add_features=['ner'])

tokens_table(corp_ner, select='sample1').tail(10)

,doc,position,token,ent_type,is_punct,is_stop,lemma,like_num,pos,tag
15,sample1,15,famous,,False,False,famous,False,ADJ,JJ
16,sample1,16,people,,False,False,people,False,NOUN,NNS
17,sample1,17,like,,False,False,like,False,ADP,IN
18,sample1,18,Missy,PERSON,False,False,Missy,False,PROPN,NNP
19,sample1,19,Elliott,PERSON,False,False,Elliott,False,PROPN,NNP
20,sample1,20,or,,False,True,or,False,CCONJ,CC
21,sample1,21,George,PERSON,False,False,George,False,PROPN,NNP
22,sample1,22,Harrison,PERSON,False,False,Harrison,False,PROPN,NNP
23,sample1,23,.,,True,False,.,False,PUNCT,.
24,sample1,24,\n,,False,False,\n,False,SPACE,_SP


A final important parameter is the `max_workers` setting. With it, you can enable parallel processing and specify the number of worker processes that can run in parallel. You should never set this value higher than the number of CPU cores in your machine (it will still work, but it will actually slow down the processing). Parallel processing makes most sense for large corpora with thousands of documents. With the small toy examples in this tutorial, you won't see performance gains (you will actually see performance loss due to increased overhead for setting up the parallel processing).

You can provide an integer value to `max_workers`, which sets the number of worker processes. 0 or 1 disable parallel processing (the default behavior).

Here, we use two workers, i.e. two CPU cores of our machine:

In [13]:
corp_2w = Corpus.from_folder('data/corpus_example', language='en', max_workers=2)
corp_2w

<Corpus [3 documents  / 2 worker processes / language "en"]>

If you use a negative number `x`, this means "use all available CPU cores, but leave `x` spare cores. For example, when your machine has four CPU cores, three of them will be used by tmtoolkit.

In [14]:
corp_allbutone = Corpus.from_folder('data/corpus_example', language='en', max_workers=-1)
corp_allbutone

<Corpus [3 documents  / 3 worker processes / language "en"]>

Finally, you can pass a float value to the `max_workers` parameter which specifies the fraction of available CPU cores to use (rounded). Here, we use 50% of the available CPU cores:

In [15]:
corp_halfcpus = Corpus.from_folder('data/corpus_example', language='en', max_workers=0.5)
corp_halfcpus

<Corpus [3 documents  / 2 worker processes / language "en"]>

In [16]:
del corp_2w, corp_allbutone, corp_halfcpus

## Corpus properties, iteration and corpus functions for document management

A `Corpus` object provides several helpful properties that summarize the text data and several corpus functions to manage the documents.

### Properties

Let's start with the number of documents in the corpus. There are two ways to obtain this value: 

In [17]:
len(corp)

3

In [18]:
corp.n_docs

3

Other important properties are the language of the documents in the corpus and the SpaCy language model used in the NLP pipeline (note that so far tmtoolkit requires that all documents within one corpus use the same language):

In [19]:
corp.language

'en'

In [20]:
corp.language_model

'en_core_web_sm'

The corpus' SpaCy NLP pipeline is given with the `nlp` property:

In [21]:
corp.nlp

As you can configure the NLP pipeline (as shown in the previous section), sentence recognition is optional. You can check if sentences were recognized using the following property:

In [22]:
corp.has_sents

True

The `max_workers` property shows how many worker processes are used during computations.

In [23]:
corp.max_workers

1

You can set new values for this property in order to temporarily enable parallel processing:

In [24]:
corp.max_workers = 2
corp

<Corpus [3 documents  / 2 worker processes / language "en"]>

In [25]:
corp.max_workers = 1   # reset

Another important property is the list of document labels:

In [26]:
corp.doc_labels

['sample1', 'sample2', 'sample3']

### Working with a `Corpus` object like with a dictionary

A `Corpus` object resembles a dictionary, i.e. it provides the same methods. The keys of this corpus-dictionary are the unique document labels and the values or elements are the respective `Document` objects.

You can access a `Corpus` object's elements via square brackets (`corp[<doc. label>]`) or `corp.get(<doc. label>)`:

In [27]:
corp['sample1']

Document "sample1" (25 tokens, 9 token attributes, 2 document attributes)

In [28]:
corp.get('sample2')

Document "sample2" (20 tokens, 9 token attributes, 2 document attributes)

The square brackets also accept an integer *i*, which will give you the document with the *i*th document label (corresponding to the order in `corp.doc_labels`):

In [29]:
corp[2]

Document "sample3" (36 tokens, 9 token attributes, 2 document attributes)

Slicing is also supported and returns a list of `Document` objects:

In [30]:
corp[0:2]

[Document "sample1" (25 tokens, 9 token attributes, 2 document attributes),
 Document "sample2" (20 tokens, 9 token attributes, 2 document attributes)]

You can also update existing documents or add new documents using the bracket syntax. For this, you can either pass a string, a `Document` object or a [SpaCy `Doc`](https://spacy.io/api/doc) object.

In [31]:
print_summary(corp)

Corpus with 3 documents in English
> sample1 (25 tokens): This is the first example file . ☺ We showcase NER...
> sample2 (20 tokens): Here comes the second example .    This one contai...
> sample3 (36 tokens): And here we go with the third and final example fi...
total number of tokens: 81 / vocabulary size: 53


In [32]:
corp['sample2'] = 'This is the updated version of the second example.'
corp['sample4'] = 'This document was added as fourth example.'
print_summary(corp)

Corpus with 4 documents in English
> sample1 (25 tokens): This is the first example file . ☺ We showcase NER...
> sample2 (10 tokens): This is the updated version of the second example ...
> sample3 (36 tokens): And here we go with the third and final example fi...
> sample4 (8 tokens): This document was added as fourth example .
total number of tokens: 79 / vocabulary size: 60


 The dictionary-interface of the `Corpus` class also allows to iterate through its contents via `corp.items()`, `corp.keys()` or `corp.values()`.

In [33]:
# iterate through pairs of document labels `lbl` and
# document objects `d` and report each document's length
for lbl, d in corp.items():
    print(f'Document length for {lbl}: {len(d)}')

Document length for sample1: 25
Document length for sample2: 10
Document length for sample3: 36
Document length for sample4: 8


In [34]:
# make a list of each document's document attributes
# (more on document attributes in the next chapter)
[d.doc_attrs for d in corp.values()]

[{'label': 'sample1', 'has_sents': True},
 {'label': 'sample2', 'has_sents': True},
 {'label': 'sample3', 'has_sents': True},
 {'label': 'sample4', 'has_sents': True}]

### Corpus functions for document management

#### Splitting documents

Sometimes you may want to split larger documents. In our example corpus, document "sample3" consists of three paragraphs:

In [35]:
from tmtoolkit.corpus import doc_texts

print(doc_texts(corp)['sample3'])

And here we go with the third and final example file.
Another line of text.

§2.
This is the second paragraph.

The third and final paragraph.


We can use the [`corpus_split_by_paragraph`](api.rst#TODO) function for splitting documents by paragraphs. There's also [`corpus_split_by_token`](api.rst#TODO) for splitting by an arbitrary token.

In [36]:
from tmtoolkit.corpus import corpus_split_by_paragraph

corpus_split_by_paragraph(corp)
print_summary(corp)

Corpus with 6 documents in English
> sample1-1 (25 tokens): This is the first example file . ☺ We showcase NER...
> sample2-1 (10 tokens): This is the updated version of the second example ...
> sample3-1 (19 tokens): And here we go with the third and final example fi...
> sample3-2 (11 tokens): § 2 .   This is the second paragraph .   
> sample3-3 (6 tokens): The third and final paragraph .
> sample4-1 (8 tokens): This document was added as fourth example .
total number of tokens: 79 / vocabulary size: 60


As we can see above, the "sample3" document was split into three individual documents, one per paragraph. You can further customize the splitting process by tweaking the parameters, e.g. the minimum number of line breaks used to detect paragraphs (default is two line breaks).

#### Joining documents

There's of course also a function for doing the inverse operation, i.e. joining several documents. This function is called [`corpus_join_documents`](api.rst#TODO) and it accepts a dictionary that maps a name for the newly joint document to a string pattern or a list of string patterns of documents to be joint. This function is especially helpful when you want to bundle lots of smaller documents (e.g. tweets) into a bigger document (e.g. all tweets of one account).

In [37]:
from tmtoolkit.corpus import corpus_join_documents

# alternative: no matching, specify each document name in a list
# corpus_join_documents(corp, {'sample3-rejoined': ['sample3-1', 'sample3-2', 'sample3-3']})

# join all documents matching "sample3*" to form a new document "sample3-rejoined"
# "sample3*" is a glob pattern, so we set this as `match_type` (more on that in the next chapter)
# we also set glue to an empty string, because we don't need to have additional line breaks
# between the documents
corpus_join_documents(corp, {'sample3-rejoined': 'sample3*'}, glue='', match_type='glob')
print_summary(corp)

Corpus with 4 documents in English
> sample1-1 (25 tokens): This is the first example file . ☺ We showcase NER...
> sample2-1 (10 tokens): This is the updated version of the second example ...
> sample4-1 (8 tokens): This document was added as fourth example .
> sample3-rejoined (36 tokens): § 2 .   This is the second paragraph .    And here...
total number of tokens: 79 / vocabulary size: 49


In [38]:
print(doc_texts(corp)['sample3-rejoined'])

§2.
This is the second paragraph.

And here we go with the third and final example file.
Another line of text.

The third and final paragraph.


### Sampling a corpus   

Finally you can sample the documents in a corpus using [Corpus.sample()](api.rst#tmtoolkit.corpus.Corpus.sample). To get a random sample of three documents from our corpus:

In [ ]:
corpus.sample(3)

In [ ]:
corpus.doc_labels

Note that this returns a new `Corpus` instance by default. You can pass `as_corpus=False` if you only need a Python dict.

The [next chapter](preprocessing.ipynb) will show how to apply several text preprocessing functions to a corpus.